In [1]:
import torch

import numpy as np
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader

from ds.drivers.datasets import DriversDataset, all_data

In [2]:
np.random.seed(1337)
torch.random.manual_seed(1337)

In [3]:
ds_all_train = DriversDataset(all_data, ds_type="train")
ds_all_test = DriversDataset(all_data, ds_type="test")

In [4]:
train_batch_size = 8
test_batch_size = 1

In [5]:
dl_train = DataLoader(
    ds_all_train,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=1,
    pin_memory=False,
    drop_last=True,
)

In [6]:
dl_test = DataLoader(
    ds_all_test,
    batch_size=test_batch_size,
    shuffle=True,
    num_workers=1,
    pin_memory=False,
    drop_last=True,
)

In [7]:
class ConvX(nn.Module):
    def __init__(self, in_planes, out_planes, kernel=3, stride=1):
        super(ConvX, self).__init__()

        self.conv = nn.Conv1d(
            in_planes,
            out_planes,
            kernel_size=kernel,
            stride=stride,
            padding=kernel // 2,
            bias=False,
        )
        self.bn = nn.BatchNorm1d(out_planes)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

In [8]:
class FCRelu(nn.Module):
    def __init__(self, in_planes, out_planes):
        super(FCRelu, self).__init__()

        self.fc = nn.Linear(in_planes, out_planes)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.fc(x))

In [9]:
class Net2Conv3FC(nn.Module):
    def __init__(self):
        super(Net2Conv3FC, self).__init__()
        self.conv_block0 = ConvX(1, 5)
        self.conv_block1 = ConvX(5, 10)
        self.fc_block0 = FCRelu(10 * 30, 100)
        self.fc_block1 = FCRelu(100, 10)
        self.final = nn.Linear(10, 2)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = self.conv_block0(x)
        x = self.conv_block1(x)
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = self.fc_block0(x)
        x = self.fc_block1(x)
        x = self.softmax(self.final(x))
        return x

In [10]:
net = Net2Conv3FC()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [11]:
def val(model):
    with torch.no_grad():
        sum_ = 0
        for i, data in enumerate(dl_test):
            inputs, labels = data
            inputs = torch.unsqueeze(inputs, 1).to(torch.float32)

            outputs = model(inputs)
            eq = outputs.max(1).indices == labels
            sum_ += eq.sum()
    return sum_

In [12]:
def train(model, epoch_count=50, print_step=50):
    min_loss = np.inf
    best_rate = 0
    for epoch in range(epoch_count):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(dl_train):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = torch.unsqueeze(inputs, 1).to(torch.float32)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % print_step == print_step - 1:
                mean_loss = running_loss / print_step
                min_loss = min(mean_loss, min_loss)
                print(f"[{epoch + 1:3d}, {i + 1:4d}] loss: {mean_loss:.3f}")
                running_loss = 0.0
        len_test = len(dl_test)
        count_of_correct = val(model)
        rate = count_of_correct / len_test
        best_rate = max(best_rate, rate)
        print(
            f"[{epoch + 1}] {count_of_correct:3d}/{len_test}; rate: {rate:.3f}"
        )
    print("Finished Training. Min_loss:", min_loss)
    return best_rate, min_loss

In [13]:
train(net, epoch_count=300)

[1,    50] loss: 0.598
[1,   100] loss: 0.576
[1,   150] loss: 0.555
[1,   200] loss: 0.532
[1,   250] loss: 0.535
[1,   300] loss: 0.525
[1] 197/639; rate: 0.308
[2,    50] loss: 0.540
[2,   100] loss: 0.500
[2,   150] loss: 0.552
[2,   200] loss: 0.530
[2,   250] loss: 0.559
[2,   300] loss: 0.507
[2] 197/639; rate: 0.308
[3,    50] loss: 0.547
[3,   100] loss: 0.522
[3,   150] loss: 0.522
[3,   200] loss: 0.554
[3,   250] loss: 0.525
[3,   300] loss: 0.499
[3] 197/639; rate: 0.308
[4,    50] loss: 0.555
[4,   100] loss: 0.511
[4,   150] loss: 0.516
[4,   200] loss: 0.535
[4,   250] loss: 0.503
[4,   300] loss: 0.529
[4] 197/639; rate: 0.308
[5,    50] loss: 0.524
[5,   100] loss: 0.511
[5,   150] loss: 0.550
[5,   200] loss: 0.527
[5,   250] loss: 0.517
[5,   300] loss: 0.513
[5] 197/639; rate: 0.308
[6,    50] loss: 0.514
[6,   100] loss: 0.511
[6,   150] loss: 0.529
[6,   200] loss: 0.479
[6,   250] loss: 0.533
[6,   300] loss: 0.512
[6] 197/639; rate: 0.308
[7,    50] loss: 0.515

(tensor(0.4726), 0.32331393122673036)